In [ ]:
!python -m ipykernel install --user --name=conda_python3
!python -m pip install metaflow

In [ ]:
%%writefile metaflow_example.py
import os
os.environ["METAFLOW_PACKAGE_SUFFIXES"] = ".py,.ipynb,.R,.RDS"

from metaflow import FlowSpec, step, batch, conda_base, conda
import random

@conda_base(libraries = {"papermill" : "2.3.3"})
class MetaflowExample(FlowSpec):
    @step
    def start(self):
        print("starting run with metaflow")
        
        # Default papermill parameters
        self.n = 10
        self.r = 20
        self.p = 0
        
        print(f"parameters evaluation\n n: {self.n}\n r: {self.r}\n p: {self.p}")
        
        self.next(self.execute_notebook_local)

    @step
    def execute_notebook_local(self):
        print("executing notebook locally")

        data = [random.randint(1, self.r) for _ in range(self.n)]
        print(f"generated data: {data}")

        self.next(self.execute_notebook_remote)
        
    @batch
    @conda(libraries = {"jupyter": "1.0.0"})
    @step
    def execute_notebook_remote(self):
        print("executing notebook remotely")

        data = [random.randint(1, self.r) for _ in range(self.n)]
        print(f"generated data: {data}")

        self.next(self.end)

    @step
    def end(self):
        print("ending run with metaflow")
 
if __name__ == '__main__':
    MetaflowExample()

In [ ]:
!python metaflow_example.py --environment=conda run